In [11]:
import os
import glob

exp_path = "/research/d5/gds/yzhong22/experiments/med_vlm_benchmark"
exp_path_mirror = "/media/yesindeed/DATADRIVE1/mount/remote_cse/experiments/med_vlm_benchmark"

In [3]:
# gemma

datasets = [
    # "slake",
    # "pathvqa",
    # "vqa-rad",
    "fairvlmed",
]

base_train_command = """llamafactory-cli train configs/2025-vlmbench/vqa/gemma3/train_{}_sft.yaml"""
base_merge_command = """llamafactory-cli export configs/2025-vlmbench/vqa/gemma3/merge_{}_sft.yaml"""

commands = []
for data in datasets:
    commands.append(base_train_command.format(data))

for data in datasets:
    commands.append(base_merge_command.format(data))

print("&&".join(commands))

llamafactory-cli train configs/2025-vlmbench/vqa/gemma3/train_fairvlmed_sft.yaml&&llamafactory-cli export configs/2025-vlmbench/vqa/gemma3/merge_fairvlmed_sft.yaml


In [7]:
# medgemma

tune_modules = ["ML"]
datasets = {
    # "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    "PathVQA": "None",
    "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_train.py \
    --peft lora --lora_r 16 --lora_alpha 32 \
    --task vqa --dataset {} \
    --model MedGemma\
    --image_path {} \
    --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it \
    --bf16 True \
    --output_dir /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --num_train_epochs 3 \
    --per_device_train_batch_size {} \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps {} \
    --save_strategy "epoch" \
    --learning_rate 2e-4 \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "linear" \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --tune_modules {}"""

per_device_train_batch_size = 4
gradient_accumulation_steps = 4

assert per_device_train_batch_size * gradient_accumulation_steps == 16

commands = []

for data, path in datasets.items():
    for tm in tune_modules:
        commands.append(
            base_command.format(
                data,
                path,
                per_device_train_batch_size,
                int(gradient_accumulation_steps),
                tm,
            )
        )

print("&&".join(commands))

python run_train.py     --peft lora --lora_r 16 --lora_alpha 32     --task vqa --dataset PathVQA     --model MedGemma    --image_path None     --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --bf16 True     --output_dir /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --num_train_epochs 3     --per_device_train_batch_size 4     --per_device_eval_batch_size 4     --gradient_accumulation_steps 4     --save_strategy "epoch"     --learning_rate 2e-4     --warmup_ratio 0.03     --lr_scheduler_type "linear"     --gradient_checkpointing True     --dataloader_num_workers 4     --tune_modules ML&&python run_train.py     --peft lora --lora_r 16 --lora_alpha 32     --task vqa --dataset VQA-RAD     --model MedGemma    --image_path None     --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --bf16 True     --output_dir /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_d

In [3]:
datasets = [
    "slake",
    "pathvqa",
    # "vqa-rad",
    "fairvlmed",
]

base_merge_command = """llamafactory-cli export configs/2025-vlmbench/vqa/medgemma/merge_{}_sft.yaml"""

commands = []

for data in datasets:
    commands.append(base_merge_command.format(data))

print("&&".join(commands))

llamafactory-cli export configs/2025-vlmbench/vqa/medgemma/merge_slake_sft.yaml&&llamafactory-cli export configs/2025-vlmbench/vqa/medgemma/merge_pathvqa_sft.yaml&&llamafactory-cli export configs/2025-vlmbench/vqa/medgemma/merge_fairvlmed_sft.yaml


# EVAL

## zero-shot

In [10]:
# gemma3

datasets = {
    # "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    # "PathVQA": "None",
    # "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_eval.py \
    --task vqa --dataset {} --split test \
    --image_path {} \
    --model Gemma3 --model_path /research/d5/gds/yzhong22/misc/pretrained/gemma-3-4b-it \
    --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --save_pred"""

commands = []
for data, path in datasets.items():

    commands.append(base_command.format(data, path))

print("&&".join(commands))

python run_eval.py     --task vqa --dataset Harvard-FairVLMed10k --split test     --image_path /research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k     --model Gemma3 --model_path /research/d5/gds/yzhong22/misc/pretrained/gemma-3-4b-it     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred


In [1]:
# medgemma

datasets = {
    "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    "PathVQA": "None",
    "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_eval.py \
    --task vqa --dataset {} --split test \
    --image_path {} \
    --model MedGemma --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it \
    --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --save_pred"""

commands = []
for data, path in datasets.items():

    commands.append(base_command.format(data, path))

print("&&".join(commands))

# print(commands[3])

python run_eval.py     --task vqa --dataset SLAKE --split test     --image_path /research/d5/gds/yzhong22/datasets/SLAKE/imgs     --model MedGemma --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&python run_eval.py     --task vqa --dataset PathVQA --split test     --image_path None     --model MedGemma --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&python run_eval.py     --task vqa --dataset VQA-RAD --split test     --image_path None     --model MedGemma --model_path /research/d5/gds/yzhong22/misc/pretrained/medgemma-4b-it     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred&&pyth

## tuned

In [11]:
# gemma3

datasets = {
    # "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    # "PathVQA": "None",
    # "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_eval.py \
    --task vqa --dataset {} --split test \
    --image_path {} \
    --model Gemma3 --model_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark/vqa/{}/gemma-3-4b-it-lora-ML-merged \
    --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --save_pred"""

commands = []
for data, path in datasets.items():

    commands.append(base_command.format(data, path, data))

print("&&".join(commands))

python run_eval.py     --task vqa --dataset Harvard-FairVLMed10k --split test     --image_path /research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k     --model Gemma3 --model_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark/vqa/Harvard-FairVLMed10k/gemma-3-4b-it-lora-ML-merged     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred


In [8]:
# medgemma

datasets = {
    # "SLAKE": "/research/d5/gds/yzhong22/datasets/SLAKE/imgs",
    # "PathVQA": "None",
    # "VQA-RAD": "None",
    "Harvard-FairVLMed10k": "/research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k",
}

base_command = """python run_eval.py \
    --task vqa --dataset {} --split test \
    --image_path {} \
    --model MedGemma --model_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark/vqa/{}/MedGemma/train_lora_ML_seed42-merged \
    --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark \
    --cache_dir /research/d5/gds/yzhong22/misc/cache \
    --save_pred"""

commands = []
for data, path in datasets.items():

    commands.append(base_command.format(data, path, data))

print("&&".join(commands))

python run_eval.py     --task vqa --dataset Harvard-FairVLMed10k --split test     --image_path /research/d5/gds/yzhong22/datasets/Harvard-FairVLMed10k     --model MedGemma --model_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark/vqa/Harvard-FairVLMed10k/MedGemma/train_lora_ML_seed42-merged     --exp_path /research/d5/gds/yzhong22/experiments/med_vlm_benchmark     --cache_dir /research/d5/gds/yzhong22/misc/cache     --save_pred
